## Imports

In [83]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from collections import Counter

## Data

In [5]:
data = pd.read_csv('./parsed_data.csv')

In [6]:
registry = pd.read_csv('./registry.csv')

## Market share

In [7]:
years = ['2020', '2021', '2022', '2023']

for year in years:
    data['market_share_' + year] = 100 * data[year] / data[year].sum()

## Age of the company

In [106]:
px.histogram(pd.DataFrame(pd.to_datetime(data['ДатаРег']).dt.year).rename(columns={'ДатаРег': 'Год'}))

In [74]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("2020 год", "2021 год", "2022 год", "2023 год"))

fig.add_trace(
    go.Scatter(x=pd.to_datetime(data['ДатаРег']).dt.year, y=data['2020'], mode='markers'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=pd.to_datetime(data['ДатаРег']).dt.year, y=data['2021'], mode='markers'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=pd.to_datetime(data['ДатаРег']).dt.year, y=data['2022'], mode='markers'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=pd.to_datetime(data['ДатаРег']).dt.year, y=data['2023'], mode='markers'),
    row=2, col=2
)

fig.update_xaxes(title_text="Год основания", row=1, col=1)
fig.update_xaxes(title_text="Год основания", row=1, col=2)
fig.update_xaxes(title_text="Год основания", row=2, col=1)
fig.update_xaxes(title_text="Год основания", row=2, col=2)

fig.update_yaxes(title_text="Объем рынка, руб", row=1, col=1)
fig.update_yaxes(title_text="Объем рынка, руб", row=1, col=2)
fig.update_yaxes(title_text="Объем рынка, руб", row=2, col=1)
fig.update_yaxes(title_text="Объем рынка, руб", row=2, col=2)

fig.show()

## Form of ownership

In [44]:
forms = pd.DataFrame(data['НаимСокр'].str.split().str[0])
forms = pd.DataFrame(forms.value_counts()[forms.value_counts() > 1]).reset_index()

In [49]:
fig = px.bar(x=forms['НаимСокр'], y=forms['count'], labels={'x': 'Форма собственности', 'y': 'Количество'})
fig.show()

## Market sizing

In [67]:
fig = px.line(x=years, y=data[years].sum(), labels={'x': 'Год', 'y': 'Общий объем рынка, руб'})
fig.show()

## Increase and decrease of market size

In [74]:
df = pd.DataFrame(data[years].sum(), columns=['sum'])
df['sum_shift'] = df['sum'].shift(1)
df['delta'] = (df['sum'] - df['sum_shift']) / df['sum_shift']

In [75]:
df.delta.mean()

0.07934062951759341

In [76]:
df

,sum,sum_shift,delta
2020,2116374075000,NaN,NaN
2021,2252469033000,2.116374e+12,0.064306
2022,2222198567000,2.252469e+12,-0.013439
2023,2638094056000,2.222199e+12,0.187155


## About AI

In [79]:
registry['Программное обеспечение относится к сфере искусственного интеллекта'].value_counts()

Программное обеспечение относится к сфере искусственного интеллекта
Да    121
Name: count, dtype: int64

## Rating by number of software

In [31]:
pd.DataFrame(registry['Идентификационный номер (ИНН)'].value_counts()
            ).join(registry[['Наименование (ФИО) правообладателя', 'Идентификационный номер (ИНН)']
                           ].set_index('Идентификационный номер (ИНН)')
                  ).reset_index().drop_duplicates(subset='Идентификационный номер (ИНН)', keep='first'
                                                 ).sort_values(by='count', ascending=False).reset_index(drop=True)[:20]

,Идентификационный номер (ИНН),count,Наименование (ФИО) правообладателя
0,2632068663,38,АО «Монитор Электрик»
1,2128050753,36,"ООО ""КС"""
2,7707083893,24,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ"""
3,7736632467,22,"АО ""СБЕРТЕХ"""
4,7707049388,16,"ПАО ""Ростелеком"""
5,5259013045,15,"ООО ""ПЕТР ТЕЛЕГИН"""
6,7728361580,15,"ООО ""ПЕРВЫЙ БИТ"""
7,7713555858,14,"ООО ""ДАТА ИНТЕГРЕЙШН СОФТВЕР"""
8,7813656659,13,ООО «УК ЮД-КАПИТАЛ»
9,7801619483,11,"ООО ""АСКОН-СИСТЕМЫ ПРОЕКТИРОВАНИЯ"""


## Software assignments

In [43]:
len(set(registry[registry['Наименование ПО'].str.contains('Cloud')
                ].index) | set(registry[registry['Наименование ПО'].str.contains('обл')].index))

21

In [ ]:
Counter(registry['Класс ПО'].str.replace(';', '').str.split('\n').dropna().sum())

## Market leaders

In [102]:
fig = go.Figure()

for idx, row in data.sort_values(by='2020', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data.sort_values(by='2021', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data.sort_values(by='2022', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )
    
fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data.sort_values(by='2023', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()

In [104]:
data1 = data.query('ОКВЭД == "Разработка компьютерного программного обеспечения"')

fig = go.Figure()

for idx, row in data1.sort_values(by='2020', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data1.sort_values(by='2021', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data1.sort_values(by='2022', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )
    
fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data1.sort_values(by='2023', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()

In [106]:
data1 = data.query('ОКВЭД == "Деятельность в области защиты информации и обеспечения безопасности критической информационной инфраструктуры"')

fig = go.Figure()

for idx, row in data1.sort_values(by='2020', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data1.sort_values(by='2021', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data1.sort_values(by='2022', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )
    
fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()
fig = go.Figure()
    
for idx, row in data1.sort_values(by='2023', ascending=False).head(10).iterrows():
    fig.add_trace(
        go.Scatter(
            x=years,
            y=row[years],
            mode="lines",
            name=row['НаимСокр'],
            showlegend=True
        )
    )

fig.update_xaxes(title_text="Год")
fig.update_yaxes(title_text="Объем рынка, руб")
fig.show()